# Lab | BabyAGI with agent

**Change the planner objective below by changing the objective and the associated prompts and potential tools and agents - Wear your creativity and AI engineering hats
You can't get this wrong!**

You would need the OpenAI API KEY and the [SerpAPI KEY](https://serpapi.com/manage-api-keyhttps://serpapi.com/manage-api-key) to run this lab.


## BabyAGI with Tools

This notebook builds on top of [baby agi](baby_agi.html), but shows how you can swap out the execution chain. The previous execution chain was just an LLM which made stuff up. By swapping it out with an agent that has access to tools, we can hopefully get real reliable information

## Install and Import Required Modules

In [1]:
from typing import Optional

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_experimental.autonomous_agents import BabyAGI
from langchain_openai import OpenAI, OpenAIEmbeddings

## Connect to the Vector Store

Depending on what vectorstore you use, this step may look different.

In [2]:
# %pip install faiss-cpu > /dev/null
# %pip install google-search-results > /dev/null
from langchain.docstore import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
SERPAPI_API_KEY = os.getenv('SERPAPI_API_KEY')

In [4]:
# Define your embedding model
embeddings_model = OpenAIEmbeddings()
# Initialize the vectorstore as empty
import faiss

embedding_size = 1536
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model.embed_query, index, InMemoryDocstore({}), {})

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


## Define the Chains

BabyAGI relies on three LLM chains:
- Task creation chain to select new tasks to add to the list
- Task prioritization chain to re-prioritize tasks
- Execution Chain to execute the tasks


NOTE: in this notebook, the Execution chain will now be an agent.

In [5]:
from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI

from langchain.agents import AgentExecutor, Tool, ZeroShotAgent
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.utilities import SerpAPIWrapper
from langchain_openai import OpenAI
import pandas as pd

# Initialize LLM
llm = OpenAI(temperature=0)

# Tool Definitions

# 1. To-Do List Generator
todo_prompt = PromptTemplate.from_template(
    "You are a planner who is an expert at coming up with a todo list for a given objective. Come up with a todo list for this objective: {objective}"
)
todo_chain = LLMChain(llm=llm, prompt=todo_prompt)

# 2. Information Search
search = SerpAPIWrapper()

# 3. Summarization
summarize_prompt = PromptTemplate.from_template(
    "Summarize the following text: {text}"
)
summarize_chain = LLMChain(llm=llm, prompt=summarize_prompt)

# 5. Sentiment Analysis
sentiment_prompt = PromptTemplate.from_template(
    "Analyze the sentiment of the following text: {text}"
)
sentiment_chain = LLMChain(llm=llm, prompt=sentiment_prompt)

# 6. Code Generation
code_gen_prompt = PromptTemplate.from_template(
    "Write a Python script to {task}."
)
code_gen_chain = LLMChain(llm=llm, prompt=code_gen_prompt)

# 7. Code Debugging
code_debug_prompt = PromptTemplate.from_template(
    "Debug the following Python code: {code}"
)
code_debug_chain = LLMChain(llm=llm, prompt=code_debug_prompt)

# 8. Knowledge Base Querying
def query_knowledge_base(question: str) -> str:
    # Mock implementation; replace with a real knowledge base query as needed
    knowledge_base = {
        "What is the capital of France?": "Paris",
        "Who wrote '1984'?": "George Orwell"
    }
    return knowledge_base.get(question, "Answer not found.")

# Aggregating Tools
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="Useful for when you need to answer questions about current events",
    ),
    Tool(
        name="TODO",
        func=todo_chain.run,
        description="Useful for when you need to come up with todo lists. Input: an objective to create a todo list for. Output: a todo list for that objective. Please be very clear what the objective is!",
    ),
    Tool(
        name="Summarize",
        func=summarize_chain.run,
        description="Useful for summarizing large amounts of text. Input: the text to summarize. Output: a summarized version of the text."
    ),
    Tool(
        name="SentimentAnalysis",
        func=sentiment_chain.run,
        description="Useful for analyzing the sentiment of a given text. Input: the text to analyze. Output: the sentiment analysis of the text."
    ),
    Tool(
        name="CodeGeneration",
        func=code_gen_chain.run,
        description="Useful for generating Python code snippets. Input: a description of what the script should do. Output: the Python code."
    ),
    Tool(
        name="CodeDebugging",
        func=code_debug_chain.run,
        description="Useful for debugging Python code. Input: the code that needs to be debugged. Output: the debugged code or suggestions."
    ),
    Tool(
        name="KnowledgeBaseQuery",
        func=query_knowledge_base,
        description="Useful for querying a predefined knowledge base. Input: a question. Output: the answer from the knowledge base."
    ),
]

# Define the Agent's Prompt
prefix = """You are an AI who performs one task based on the following objective: {objective}. Take into account these previously completed tasks: {context}."""
suffix = """Question: {task}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["objective", "task", "context", "agent_scratchpad"],
)

In [6]:
llm = OpenAI(temperature=0)
llm_chain = LLMChain(llm=llm, prompt=prompt)
tool_names = [tool.name for tool in tools]
agent = ZeroShotAgent(llm_chain=llm_chain, allowed_tools=tool_names)
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain.agents.mrkl.base.ZeroShotAgent` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use create_react_agent instead.
  warn_deprecated(


### Run the BabyAGI

Now it's time to create the BabyAGI controller and watch it try to accomplish your objective.

In [7]:
OBJECTIVE = "List of the top 30 selling artists of the 90's and their selling records."

In [8]:
# Logging of LLMChains
verbose = False
# If None, will keep on going forever
max_iterations: Optional[int] = 10
baby_agi = BabyAGI.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    task_execution_chain=agent_executor,
    verbose=verbose,
    max_iterations=max_iterations,
)

In [9]:
baby_agi({"objective": OBJECTIVE})

/opt/anaconda3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(



*****TASK LIST*****

1: Make a todo list

*****NEXT TASK*****

1: Make a todo list


> Entering new AgentExecutor chain...
Thought: I should ask for the objective to create a todo list for
Action: TODO
Action Input: Objective
Observation: :

"Plan a successful birthday party for a friend"

1. Set a budget: Determine how much you are willing to spend on the party and stick to it.

2. Choose a date and time: Consult with the friend and pick a date and time that works for them and any other important guests.

3. Decide on a theme: Consider the friend's interests and choose a theme that they will enjoy.

4. Create a guest list: Make a list of all the people you want to invite to the party.

5. Send out invitations: Send out invitations at least 2-3 weeks in advance to give guests enough time to RSVP.

6. Plan the menu: Decide on the food and drinks you will serve at the party. Consider any dietary restrictions of the guests.

7. Arrange for entertainment: Whether it's hiring a DJ or plann

{'objective': "List of the top 30 selling artists of the 90's and their selling records."}

I would love to go on but the kernel has died... :(